# Motivation
论文原文：[《Fast unfolding of communities in large networks》](https://arxiv.org/pdf/0803.0476)

参考学习地址：
* [louvain算法](https://zhuanlan.zhihu.com/p/178790546)
* [万物皆网络，万字长文详解社区发现算法Louvain](https://zhuanlan.zhihu.com/p/556291759)

代码库：https://github.com/taynaud/python-louvain

社区发现（Community Detection）目的在图中找到一些“潜在的有特定关系的组织”，也即社区。
* 团伙挖掘过程中，介质的质量、边的构建思路、以及业务的抽象能力，比算法更重要
* 社区发现钱一定要进行数据降噪（聚集的结构“有意思”，指代聚集本身可以翻译为一定的上层业务场景的表现）

## 算法流程
Louvain本质是一个贪心算法。
1. 将图中每个节点视为一个独立的社区，社区的数目等于节点的个数
2. 对每个节点$i$，依次尝试把节点$i$分配到其每个邻居节点所在的社区，计算分配前后的模块度变化$\Delta Q$，记录最大的邻居节点。如果$\Delta Q > 0$则将节点$i$分配给那个邻居节点所在的社区，否则保持不变
3. 重复2.，直至所有节点所属社区不再变化
4. 进行图压缩，将所有同一社区的节点压缩为一个新节点，社区内节点之间的边权重转化为新节点的环权重，社区间的边权重转化为新节点之间的边权重
5. 重复1.，直至整个图的模块度不再变化


## 模块度
|Ref.|Modularity|Directed|Weighted|Community Type|
|:---|:---------|:-------|:-------|:-------|
||||||
|Newman 2004a|$Q=\frac{1}{2m} \sum_{i,j} [A_{ij}−\frac{k_i k_j}{2m}]\delta (C_i,C_j )$|No|No|Disjoint|
|Newman & Girvan 2004|$Q=\sum_{k=1}^{s}[\frac{l_k}{L}-(\frac{d_k}{2L})^2]$|No|No|Disjoint|
|Newman 2004a|$Q=\frac{1}{2W}\sum_{i,j}[A_{i,j}-\frac{s_is_j}{2W}]\delta(C_i,C_j)$|No|Yes|Disjoint|
|Fortunato 2010|$Q=\sum_{c=1}^{n_c}[\frac{W_c}{W}-(\frac{S_c}{2W})^2]$|No|Yes|Disjoint|
|Arenas et al. 2007|$Q=\frac{1}{2m}\sum_{i,j}[A_{ij}-\frac{k_i^{out}k_j^{in}}{m}]\delta(C_i,C_j)$|Yes|No|Disjoint|
|Arenas et al. 2007|$Q=\frac{1}{2W}\sum_{i,j}[W_{ij}-\frac{k_i^{out}k_j^{in}}{W}]\delta(C_i,C_j)$|Yes|Yes|Disjoint|
|Shen et al. 2009|$Q=\frac{1}{2m}\sum_{ij}\frac{1}{O_iO_j}(A_{ij}-\frac{k_ik_j}{2m})\delta(C_i,C_j)$|No|No|Overlapping|
|Nicosia et al. 2009|$Q=\frac{1}{m}\sum_{c=1}^{n_c}\sum_{i,j}[r_{ijc}A_{ij}-s_{ijc}\frac{k_i^{out}k_j^{in}}{m}]$|Yes|No|Overlapping|

Disjoint表示社区不重叠，overlapping表示社区存在重叠

物理含义：社团内，实际的边数与随机分布情况（相同节点度分布的随机网络）下的边数差距。如果差距大，说明社团内部聚集程度显著高于随机情况，社团划分质量好。模块度的取值范围在 [-0.5, 1] 之间，在[0.3, 0.7]之间为合适的社区划分结果。

$$
\begin{align}
Q=\frac{1}{2m} \sum_{i,j} [A_{ij}−\frac{k_i k_j}{2m}]\delta (C_i,C_j ) \text{,  where } \delta(u,v)=
\begin{cases}1 & \text{, if } u == v \\
0 & \text{ otherwise.}
\end{cases}
\end{align}
$$
* $A_{ij}$为节点$i$和节点$j$之间边的权重
* $m$是所有边权重之和, i.e. $m=\frac{1}{2}\sum_{ij}A_{ij}$
* $k_i=\sum_jA_{ij}$为所有和节点$i$连接的边权重之和
* $c_i$表示节点$i$所属的社区
* $A_{ij}-\frac{k_ik_j}{2m}=A_{ij}-k_i\frac{k_j}{2m}$其中$\frac{k_j}{2m}$可以看作是节点$j$连接至任一节点的概率，$k_i\frac{k_j}{2m}$可以解释为随机情况下节点$i$和节点$j$的边
  
Louvain默认采用无权无向图，和实际应用中多为有权有向图不同，模块度计算如下：
$$
\begin{align}
Q &= \frac{1}{2m}\sum_{ij}[A_{ij}-\frac{k_ik_j}{2m}]\delta(C_i,C_j) \\
&= \frac{1}{2m}[\sum_{ij}A_{ij}-\frac{\sum_ik_i\sum_jk_j}{2m}]\delta(C_i,C_j) \\
&= \frac{1}{2m}\sum_c[\sum in -\frac{(\sum tot)^2}{2m}] \\
&= \sum_c[e_c-a_c^2]
\end{align}
$$
注意：该模块度计算和kmeans中sse的目标函数类似，是针对于全体数据的一种目标函数。kmeans希望sse越小越好，而louvain希望modularity越大越好。$s$表示有$c$个社区，$e_c$表示第$i$个社区内部节点之间的权重之和，$a_c$表示这个社区和外界连接的权重之和。

## 模块度增益
$$
\begin{align}
\Delta Q(\underbrace{i \rightarrow C}_{节点i加入社区C}) &=\underbrace{[\frac{\sum_{in}+k_{i,in}}{2m}-(\frac{\sum_{tot}+k_i}{2m})^2]}_{节点i加入社区C之后社区C的局部模块度} - \underbrace{[\frac{\sum_{in}}{2m} - (\frac{\sum_{tot}}{2m})^2 - (\frac{k_i}{2m})^2]}_{节点加入社区C之前，社区C的局部模块度和节点i原社区的模块度之和} \\
 &= \frac{1}{2m}(k_{i,in}-\frac{\sum_{tot}k_i}{m}) 
\end{align}
$$
模块度增益是针对单个节点（社区）定义的，当某个节点（社区）合并至某个社区中，合并前后整体图模块度的差异
* $\sum_{in}$ 是社区$C$内部连接权重之和
* $k_{i,in}$是节点$i$和社区$C$之间的连接权重之和
* $\sum_{tot}$是社区$C$所有连接权重的和，包括社区与外部的链接
* $k_i$是与节点$i$相关的所有链接权重之和

```python
def modularity(G, communities, weight="weight", resolution=1):
    Parameters
    ----------
    G : NetworkX Graph
    communities : list or iterable of set of nodes
        These node sets must represent a partition of G's nodes.
    weight : string or None, optional (default="weight")
            The edge attribute that holds the numerical value used
            as a weight. If None or an edge does not have that attribute,
            then that edge has weight 1.
    resolution : float (default=1)
        If resolution is less than 1, modularity favors larger communities.
        Greater than 1 favors smaller communities.
    Returns
    -------
    Q : float
        The modularity of the paritition.
    if not isinstance(communities, list):
        communities = list(communities)
    if not is_partition(G, communities):
        raise NotAPartition(G, communities)
    directed = G.is_directed()
    if directed:
        out_degree = dict(G.out_degree(weight=weight))
        in_degree = dict(G.in_degree(weight=weight))
        m = sum(out_degree.values())
        norm = 1 / m ** 2
    else:
        out_degree = in_degree = dict(G.degree(weight=weight))
        deg_sum = sum(out_degree.values())
        m = deg_sum / 2
        norm = 1 / deg_sum ** 2
    def community_contribution(community):
        comm = set(community)
        L_c = sum(wt for u, v, wt in G.edges(comm, data=weight, default=1) if v in comm)
        out_degree_sum = sum(out_degree[u] for u in comm)
        in_degree_sum = sum(in_degree[u] for u in comm) if directed else out_degree_sum
        return L_c / m - resolution * out_degree_sum * in_degree_sum * norm
    return sum(map(community_contribution, communities))
```

1) 在工程实现上，通常将所有图当作有向图，因为无向图本质是等权的双向图；
2) 分别统计整个graph的加权出入度，也即一个节点所在的所有指定方向边的权重之和；
3) 统计整个图中所有节点的加权出度和m并计算m的平方的倒数
4) 分别统计每个社区中两个方向边的权重之和
5) 分别统计每个社区中所有节点的加权出度之和以及所有节点加权入度之和
6) 根据公式 L_c / m - resolution * out_degree_sum * in_degree_sum * norm 计算每个社区的模块度

### 示例

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/louvain-example.png" width = "600" height = "200" alt="Louvain示例" align=center />

初始关系如上图，以节点A为例，节点之间合并时（以$A\rightarrow B$为例）有$Q_{AB}=5-\frac{10*7}{30}=2.7$。其中
* 5为AB之间的权重
* 30为所有边权重 30=5+2+4+1+7+8+3
* 10为A的所有连接边权重 10=5+4+1
* 7为其他节点和社区B之间的权重 7=5+2
对比原公式$\Delta Q=\frac{1}{2m}(k_{i,in}-\frac{\sum_{tot}k_i}{m})$，其中$k_{i,in}$为A，$k_i$为B。